# Cine 21 영화배우 정보 크롤링
- [site 주소](http://www.cine21.com/rank/person)
- 개발자 도구에서 xhr 인 것만 필터링 network -> content 페이지의 요청 방식 확인
- [Request url](http://www.cine21.com/rank/person/content)
- Request Method : Post
- Form Data (1month data)
    - section: actor
    - period_start: 2020-07
    - gender: all
    - page: 1
- 1~2 년치 데이터를 한번에 크롤링 해서 actors.json 파일로 저장하기

# 1단계

In [44]:
import requests
from bs4 import BeautifulSoup
import re
import pymongo
from urllib.parse import urljoin

#  정규표현식에서 자르기 사용
- \w 는 문자와 숫자
- *(asterisk): 0~n 횟수, +: 1~n, ? : 0 or 1

- .(period): 줄바꿈 문자 (\n)을 제외한 모든 글자 1개를 의미한다.
- * 는 문자가 0번 또는 그 이상 반복되는 패턴
- Greedy (.\*) vs Non-Greedy(.\*?)
    - Greedy (.\*): 모두 선택됨
    - Non-Greedy (.\*?): 첫번째 매칭되는 태그만 선택됨
- [\n\r\t]: 요소 중 하나를 찾는다.
- re.compile()는 해당 정규표현식을 찾아주는 컴파일러를 생성해주는 함수
- compile 함수내의 정규표현식 앞에 항상 r을 붙여주어야 한다.


```
test_data = '유아인(2편)'
re.sub('\(\w*\)','',test_data)
```


In [91]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dic = dict()
formdata_dic['section'] = 'actor'
formdata_dic['period_start'] = '2020-07' 
formdata_dic['gender'] = 'all'
formdata_dic['page'] = 1

res = requests.post(actor_url, data=formdata_dic)
soup = BeautifulSoup(res.text, 'html.parser')
for actor in soup.select('li.people_li div.name'):
#     print(actor.text)
    re.sub('\(\w*\)','',actor.text)

actor_item_list = list()
for a_tag in soup.select('li.people_li div.name a'):
#     print(a_tag.attrs.get('href')) # print(a_tag['href'])
    actor_detail_url = urljoin(actor_url, a_tag['href'])
    
#     정보를 가진 url 링크 확인
#     print(actor_detail_url)
    res = requests.get(actor_detail_url)
    
#     정보 url에 요청했을때 응답 200 확인
#     print(res.status_code)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    
    actor_item_dic = dict()
    
    for li_tag in soup.select('ul.default_info li'):
#         print(li_tag)
#         print(li_tag.text)
#         dic의 key에 해당하는 값을 추출한다. dict['직업']
        actor_item_field = li_tag.select_one('span.tit').text # 직업
    
#         print(actor_item_field)
#         dict의 value에 해당하는 값을 추출한다. dict['직업'] = '배우'
#         <li><span class= 'tit'>직업</span>배우<li>
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag))
        actor_item_value= re.sub('<.*?>','',actor_item_value) #배우
        regex = re.compile(r'[\n\r\t]') # [ ] 요소중에 하나
        actor_item_value = regex.sub('',actor_item_value)
        actor_item_dic[actor_item_field] = actor_item_value
    actor_item_list.append(actor_item_dic)
# print(len(actor_item_list))
    

7


In [81]:
test_data = '''
여

https://www.facebook.com/leere.vv/

배우
1965-11-06
'''
regex = re.compile(r'[\n\r\t]') # [ ] 요소중에 하나
regex.sub('',test_data)

'여https://www.facebook.com/leere.vv/배우1965-11-06'

# 2단계
- 1 page의 7명의 배우의 정보를 출력

In [105]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dic = dict()
formdata_dic['section'] = 'actor'
formdata_dic['period_start'] = '2020-07' 
formdata_dic['gender'] = 'all'
formdata_dic['page'] = 1

res = requests.post(actor_url, data=formdata_dic)
soup = BeautifulSoup(res.text, 'html.parser')

actor_item_list = list()

actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
rankings = soup.select('li.people_li > span.grade')
for index, actor in enumerate(actors):
    actor_item_dic = dict()
    
#     유아인(2편)에서 (2편)을 제거하고 저장한다.
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_item_dic['배우이름'] = actor_name
    
#     흥행지수 값에서 콤마를 제거하고 정수타입으로 변환해서 저장한다.
    actor_hits = int(hits[index].text.replace(',',''))
    actor_item_dic['흥행지수'] = actor_hits

#     출연작 
    movie_titles = movies[index].select('li a span')

#     출연자 여러개의 title을 저장하는 리스트
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    actor_item_dic['출연영화'] = movie_title_list

#     순위
    actor_ranking = rankings[index].text
    actor_item_dic['랭킹'] = actor_ranking
        
#     print(actor)
#     print(actor_name)
#     print(actor_hits)
#     print(movie_titles)
#     print(movie_title_list)
#     print(actor_ranking)


    # 배우의 상세정보를 보기 위해 다시 requests
    actor_detail_url = actor.select_one('a').attrs['href']
    actor_detail_full_url = urljoin(actor_url, actor_detail_url)
#     print(actor_detail_full_url)
    
#     정보를 가진 url 링크 확인
#     print(actor_detail_url)
    res = requests.get(actor_detail_full_url)
    
#     정보 url에 요청했을때 응답 200 확인
#     print(res.status_code)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    
    for li_tag in soup.select('ul.default_info li'):
#         print(li_tag)
#         print(li_tag.text)
#         dic의 key에 해당하는 값을 추출한다. dict['직업']
        actor_item_field = li_tag.select_one('span.tit').text # 직업
    
#         print(actor_item_field)
#         dict의 value에 해당하는 값을 추출한다. dict['직업'] = '배우'
#         <li><span class= 'tit'>직업</span>배우<li>
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag))
        actor_item_value= re.sub('<.*?>','',actor_item_value) #배우
        regex = re.compile(r'[\n\r\t]') # [ ] 요소중에 하나
        actor_item_value = regex.sub('',actor_item_value)
        actor_item_dic[actor_item_field] = actor_item_value
    actor_item_list.append(actor_item_dic)
    

actor_item_list
    

[{'배우이름': '강동원',
  '흥행지수': 159665,
  '출연영화': ['반도'],
  '랭킹': '1',
  '직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'배우이름': '이정현',
  '흥행지수': 139707,
  '출연영화': ['반도'],
  '랭킹': '2',
  '직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'배우이름': '이레',
  '흥행지수': 119748,
  '출연영화': ['반도'],
  '랭킹': '3',
  '직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'배우이름': '권해효',
  '흥행지수': 99795,
  '출연영화': ['반도', '사이비', '한낮의 피크닉'],
  '랭킹': '4',
  '직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'배우이름': '구교환',
  '흥행지수': 79832,
  '출연영화': ['반도'],

#  3단계
- 1, 2, 3 페이지의 21명 배우의 정보를 출력

In [108]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dic = dict()
formdata_dic['section'] = 'actor'
formdata_dic['period_start'] = '2020-07' 
formdata_dic['gender'] = 'all'
actor_item_list = list()

for page in range(1, 4):
    formdata_dic['page'] = page
    
    print('--->{} 페이지 출력됨'.format(page))
    
    res = requests.post(actor_url, data=formdata_dic)
    soup = BeautifulSoup(res.text, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    for index, actor in enumerate(actors):
        print('===> {} 번째 배우 출력됨.'.format(index))
        actor_item_dic = dict()

    #     유아인(2편)에서 (2편)을 제거하고 저장한다.
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_item_dic['배우이름'] = actor_name

    #     흥행지수 값에서 콤마를 제거하고 정수타입으로 변환해서 저장한다.
        actor_hits = int(hits[index].text.replace(',',''))
        actor_item_dic['흥행지수'] = actor_hits

    #     출연작 
        movie_titles = movies[index].select('li a span')

    #     출연자 여러개의 title을 저장하는 리스트
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_item_dic['출연영화'] = movie_title_list

    #     순위
        actor_ranking = rankings[index].text
        actor_item_dic['랭킹'] = actor_ranking

        # 배우의 상세정보를 보기 위해 다시 requests
        actor_detail_url = actor.select_one('a').attrs['href']
        actor_detail_full_url = urljoin(actor_url, actor_detail_url)

    #     정보를 가진 url 링크 확인
        res = requests.get(actor_detail_full_url)

        soup = BeautifulSoup(res.text, 'html.parser')

        for li_tag in soup.select('ul.default_info li'):

    #         dic의 key에 해당하는 값을 추출한다. dict['직업']
            actor_item_field = li_tag.select_one('span.tit').text # 직업


    #         dict의 value에 해당하는 값을 추출한다. dict['직업'] = '배우'
    #         <li><span class= 'tit'>직업</span>배우<li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag))
            actor_item_value= re.sub('<.*?>','',actor_item_value) #배우
            regex = re.compile(r'[\n\r\t]') # [ ] 요소중에 하나
            actor_item_value = regex.sub('',actor_item_value)
            actor_item_dic[actor_item_field] = actor_item_value
        actor_item_list.append(actor_item_dic)


print(len(actor_item_list))

--->1 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->2 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->3 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
21


# 4단계
- 1905 명의  1년치 데이터 정보를 가져옴

In [114]:
import requests
from bs4 import BeautifulSoup
import re
import pymongo
from urllib.parse import urljoin
from itertools import count

actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dic = dict()
formdata_dic['section'] = 'actor'
formdata_dic['period_start'] = '2019-07' 
formdata_dic['gender'] = 'all'
actor_item_list = list()

for page in count(1): # 1부터 무한루프 돌아라
    formdata_dic['page'] = page
    
    print('--->{} 페이지 출력됨'.format(page))
    
    res = requests.post(actor_url, data=formdata_dic)
    soup = BeautifulSoup(res.text, 'html.parser')

    actors = soup.select('li.people_li div.name')    
    if len(actors) == 0:
        break
        
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    for index, actor in enumerate(actors):
        print('===> {} 번째 배우 출력됨.'.format(index))
        actor_item_dic = dict()

    #     유아인(2편)에서 (2편)을 제거하고 저장한다.
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_item_dic['배우이름'] = actor_name

    #     흥행지수 값에서 콤마를 제거하고 정수타입으로 변환해서 저장한다.
        actor_hits = int(hits[index].text.replace(',',''))
        actor_item_dic['흥행지수'] = actor_hits

    #     출연작 
        movie_titles = movies[index].select('li a span')

    #     출연자 여러개의 title을 저장하는 리스트
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_item_dic['출연영화'] = movie_title_list

    #     순위
        actor_ranking = rankings[index].text
        actor_item_dic['랭킹'] = actor_ranking

        # 배우의 상세정보를 보기 위해 다시 requests
        actor_detail_url = actor.select_one('a').attrs['href']
        actor_detail_full_url = urljoin(actor_url, actor_detail_url)

    #     정보를 가진 url 링크 확인
        res = requests.get(actor_detail_full_url)

        soup = BeautifulSoup(res.text, 'html.parser')

        for li_tag in soup.select('ul.default_info li'):

    #         dic의 key에 해당하는 값을 추출한다. dict['직업']
            actor_item_field = li_tag.select_one('span.tit').text # 직업


    #         dict의 value에 해당하는 값을 추출한다. dict['직업'] = '배우'
    #         <li><span class= 'tit'>직업</span>배우<li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag))
            actor_item_value= re.sub('<.*?>','',actor_item_value) #배우
            regex = re.compile(r'[\n\r\t]') # [ ] 요소중에 하나
            actor_item_value = regex.sub('',actor_item_value)
            actor_item_dic[actor_item_field] = actor_item_value
        actor_item_list.append(actor_item_dic)

print(len(actor_item_list))

--->1 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->2 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->3 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->4 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->5 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->6 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->7 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->8 페이지 출력됨
===> 0

===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->60 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->61 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->62 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->63 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->64 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->65 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->66 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 

===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->118 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->119 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->120 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->121 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->122 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->123 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->124 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1

===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->176 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->177 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->178 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->179 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->180 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->181 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->182 페이지 출력됨
===> 0

--->233 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->234 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->235 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->236 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->237 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->238 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->239 페이지 출력됨
===> 0 번째 배우 출력됨.
===> 1 번째 배우 출력됨.
===> 2 번째 배우 출력됨.
===> 3 번째 배우 출력됨.
===> 4 번째 배우 출력됨.
===> 5 번째 배우 출력됨.
===> 6 번째 배우 출력됨.
--->24

In [115]:
print(len(actor_item_list))
actor_final_list = [ actor for actor in actor_item_list if len(actor['배우이름']) < 10 ]
print(len(actor_final_list))


1905
1903
